В рамках выполнения данного задания вам необходимо создать панораму вашей комнаты. Панорама должна состоять минимум из 5 изображений. Они должны быть сняты таким образом, чтобы любое из них имело пересечение не более, чем с двумя другими

Баллы будут проставлены следующим образом:
* До 7 баллов за построение панорамы (зависит от её итогового качества)
* 3 балла за самостоятельную имплементацию любого из рассмотренных на лекции или семинаре алгоритма

In [1]:
!pip install opencv-python matplotlib numpy ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00


In [2]:
import glob
import itertools
import math
import os.path as osp
import time

import cv2
import matplotlib
import numpy as np

#matplotlib.use('TkAgg')  # for macOS

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 16.0)
plt.rcParams['image.interpolation'] = 'bilinear'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2
%load_ext autotime

time: 1 ms (started: 2024-03-21 18:52:37 +00:00)


# Panorama stitching

## Load Images

In [ ]:
input_dir = 'input'
ipaths = sorted(glob.glob(osp.join(input_dir, 'img_*.jpg')))
n_imgs = len(ipaths)

images_info = dict()
for i_num, ipath in enumerate(ipaths):
    img = plt.imread(ipath)
    images_info[ipath] = {'img': img}
    plt.subplot(1, n_imgs, i_num + 1)
    plt.title(osp.basename(ipath), fontsize=20)
    plt.axis('off')
    plt.imshow(images_info[ipath]['img'])
plt.show()

## Keypoints detection + descriptors

In [ ]:
def keypoints_detection_sift(input_img):
    gray = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT_create()
    kps, dscrs = sift.detectAndCompute(gray, None) #keypoints and descriptors
    return kps, dscrs

time: 1.14 ms (started: 2024-03-20 18:54:09 +00:00)


In [ ]:
for i_num, ipath in enumerate(images_info):
    img = images_info[ipath]['img']
    keypoints, descriptors = keypoints_detection_sift(img)
    images_info[ipath]['keypoints'] = keypoints
    images_info[ipath]['descriptors'] = descriptors
    plt.subplot(1, n_imgs, i_num + 1)
    plt.title(osp.basename(ipaths[i_num]), fontsize=20)
    plt.imshow(cv2.drawKeypoints(
        img, keypoints, None,
        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    ))
    plt.axis('off')
plt.show()

In [ ]:
keypoints, descriptors = images_info[ipaths[0]]['keypoints'], \
    images_info[ipaths[0]]['descriptors']
keypoint = sorted(keypoints, key=lambda x: x.size, reverse=True)[0]
for field in dir(keypoint):
    if not field.startswith('_'):
        print(f'{field:10} {getattr(keypoint, field)}')

angle      38.66455078125
class_id   -1
convert    <built-in method convert of type object at 0x56df07b93300>
octave     5440262
overlap    <built-in method overlap of type object at 0x56df07b93300>
pt         (825.1912231445312, 1027.3291015625)
response   0.05842962488532066
size       393.2807312011719
time: 15 ms (started: 2024-03-20 18:56:02 +00:00)


In [ ]:
_ = plt.figure(figsize=(10, 8))
plt.imshow(cv2.drawKeypoints(
    images_info[ipaths[0]]['img'], [keypoint, ], None,
    flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
)
plt.axis('off')
plt.show()

## Keypoints matching

### Bruteforce + Cross-check

In [ ]:
def keypoints_matching_cross_check(dscrs1, dscrs2):
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    cross_matches = bf.match(dscrs1, dscrs2)
    return cross_matches

time: 1.09 ms (started: 2024-03-20 18:56:29 +00:00)


In [ ]:
matches = keypoints_matching_cross_check(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors'],

)

time: 3.87 s (started: 2024-03-20 18:56:31 +00:00)


In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

### KNN + Ratio test

In [ ]:
def keypoints_matching_knn(dscrs1, dscrs2):
    bf = cv2.BFMatcher()
    tic = time.time_ns()
    knn_matches = bf.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    print(f'Matching time: {(toc - tic) / 10e6 :.5f} ms')
    good_matches = []
    for neighbour_1, neighbour_2 in knn_matches:
        if neighbour_1.distance < 0.6 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

time: 1.33 ms (started: 2024-03-20 18:56:49 +00:00)


In [ ]:
matches = keypoints_matching_knn(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

Matching time: 275.49198 ms
time: 2.76 s (started: 2024-03-20 18:56:52 +00:00)


In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

### FLANN

In [ ]:
def keypoints_matching_flann(dscrs1, dscrs2):
    index_params = dict(
        algorithm=1,  # FLANN_INDEX_KDTREE
        trees=5
    )
    search_params = dict(checks=20)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    tic = time.time_ns()
    flann_matches = flann.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    print(f'Matching time: {(toc - tic) / 10e6 :.5f} ms')
    good_matches = []
    for neighbour_1, neighbour_2 in flann_matches:
        if neighbour_1.distance < 0.6 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

time: 1.34 ms (started: 2024-03-20 18:57:08 +00:00)


In [ ]:
matches = keypoints_matching_flann(
    images_info[ipaths[0]]['descriptors'],
    images_info[ipaths[1]]['descriptors']
)

Matching time: 21.10247 ms
time: 215 ms (started: 2024-03-20 18:57:11 +00:00)


In [ ]:
img_matches = cv2.drawMatches(
    images_info[ipaths[0]]['img'],
    images_info[ipaths[0]]['keypoints'],
    images_info[ipaths[1]]['img'],
    images_info[ipaths[1]]['keypoints'],
    matches,
    None,  # output image
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
plt.imshow(img_matches)
plt.axis('off')
plt.show()

In [ ]:
print(f'Descriptors shape on the image_0: {images_info[ipaths[0]]["descriptors"].shape}')
print(f'Descriptors shape on the image_1: {images_info[ipaths[1]]["descriptors"].shape}')
mock_descriptors_1 = np.repeat(images_info[ipaths[0]]['descriptors'], 100, axis=0)
mock_descriptors_2 = np.repeat(images_info[ipaths[1]]['descriptors'], 100, axis=0)
print(f'Descriptors shape on the mock_image_0: {mock_descriptors_1.shape}')
print(f'Descriptors shape on the mock_image_1: {mock_descriptors_2.shape}')

Descriptors shape on the image_0: (12810, 128)
Descriptors shape on the image_1: (5701, 128)
Descriptors shape on the mock_image_0: (1281000, 128)
Descriptors shape on the mock_image_1: (570100, 128)
time: 318 ms (started: 2024-03-20 18:57:41 +00:00)


In [ ]:
_ = keypoints_matching_knn(mock_descriptors_1, mock_descriptors_2)

error: OpenCV(4.8.0) /io/opencv/modules/features2d/src/matchers.cpp:860: error: (-215:Assertion failed) trainDescCollection[iIdx].rows < IMGIDX_ONE in function 'knnMatchImpl'


time: 13.5 ms (started: 2024-03-20 18:57:43 +00:00)


In [ ]:
_ = keypoints_matching_flann(mock_descriptors_1, mock_descriptors_2)

Matching time: 1812.39788 ms
time: 18.4 s (started: 2024-03-20 18:58:02 +00:00)


### Find matches for images

In [ ]:
matches_images = dict()
for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
    matches_pair = keypoints_matching_knn(
        images_info[ipath_0]['descriptors'],
        images_info[ipath_1]['descriptors']
    )
    matches_images[(ipath_0, ipath_1)] = matches_pair

Matching time: 191.52193 ms
Matching time: 144.45000 ms
Matching time: 175.31976 ms
Matching time: 408.19504 ms
Matching time: 215.51379 ms
Matching time: 63.21699 ms
Matching time: 79.17871 ms
Matching time: 129.41532 ms
Matching time: 169.79074 ms
Matching time: 63.68654 ms
Matching time: 60.49879 ms
Matching time: 97.77246 ms
Matching time: 184.15968 ms
Matching time: 131.15187 ms
Matching time: 97.27760 ms
Matching time: 153.81051 ms
Matching time: 290.76884 ms
Matching time: 125.73400 ms
Matching time: 95.38238 ms
Matching time: 116.57554 ms
time: 30 s (started: 2024-03-20 18:58:22 +00:00)


## Homography

In [ ]:
homographies = dict()
for (ipath_0, ipath_1), matches in matches_images.items():
    src_pts = np.float32(
        [images_info[ipath_0]['keypoints'][m.queryIdx].pt for m in matches]
    ).reshape(-1, 1, 2)
    dst_pts = np.float32(
        [images_info[ipath_1]['keypoints'][m.trainIdx].pt for m in matches]
    ).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(
        src_pts,
        dst_pts,
        cv2.RANSAC,
        ransacReprojThreshold=.5,
        maxIters=20000,
        confidence=0.995
    )
    homographies[ipath_0, ipath_1] = (M, mask.ravel().tolist())

time: 2.46 s (started: 2024-03-20 18:59:02 +00:00)


In [ ]:
M

array([[ 4.66663641e-01, -3.98567112e-02,  2.20489699e+03],
       [-3.23201188e-01,  9.37914836e-01,  1.68369060e+02],
       [-1.93175761e-04,  1.05124982e-05,  1.00000000e+00]])

time: 6.78 ms (started: 2024-03-20 18:59:08 +00:00)


In [ ]:
for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
    matches = matches_images[ipath_0, ipath_1]
    M, matches_mask = homographies[ipath_0, ipath_1]
    img_0, img_1_polylines = images_info[ipath_0]['img'].copy(), \
        images_info[ipath_1]['img'].copy()
    rows_0, cols_0 = img_0.shape[:2]
    rows_1, cols_1 = img_1_polylines.shape[:2]
    pts = np.float32(
        [[0, 0], [0, rows_0], [cols_0, rows_0], [cols_0, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    img_1_polylines = cv2.polylines(
        img_1_polylines, [np.int32(dst)], True, 255, 3, cv2.LINE_AA #полигон наложения
    )
    img_matches = cv2.drawMatches(
        img_0,
        images_info[ipath_0]['keypoints'],
        img_1_polylines,
        images_info[ipath_1]['keypoints'],
        matches,
        None,  # output image
        matchesMask=matches_mask,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )
    plt.title(
        f'Pair of images: {ipath_0} and {ipath_1} ({sum(matches_mask)} matches)',
        fontsize=20
    )
    plt.imshow(img_matches)
    plt.axis('off')
    plt.show()

## Stitching images

In [ ]:
M, _ = homographies[ipaths[1], ipaths[0]]
img_0, img_1 = images_info[ipaths[0]]['img'], images_info[ipaths[1]]['img']
img_0_1 = cv2.warpPerspective(
    img_1, M, (img_0.shape[1] + img_1.shape[1], img_1.shape[0])
)
img_0_1[0:img_0.shape[0], 0:img_0.shape[1]] = img_0

# Some postprocessing
img_0_1_mask = np.where(img_0_1.sum(axis=2) != 0, 1, 0).astype('uint8')
kernel = np.ones((5, 5), np.uint8)
img_0_1_mask = cv2.morphologyEx(img_0_1_mask, cv2.MORPH_CLOSE, kernel)
img_0_1_mask = cv2.erode(img_0_1_mask, kernel, iterations=1)
img_0_1_mask = np.dstack([img_0_1_mask, ] * 3)

time: 1.09 s (started: 2024-03-20 19:01:11 +00:00)


In [ ]:
plt.imshow(img_0_1)
plt.axis('off')
plt.show()

In [ ]:
M, _ = homographies[ipaths[2], ipaths[0]]
img_2 = images_info[ipaths[2]]['img']
img_2_pano = cv2.warpPerspective(
    img_2, M, (img_0.shape[1] + img_1.shape[1], img_1.shape[0])
)
panorama = np.where(img_0_1_mask != 0, img_0_1, img_2_pano)

time: 443 ms (started: 2024-03-20 19:01:47 +00:00)


In [ ]:
plt.imshow(panorama)
plt.show()

# Homework

In [71]:
for i in range(5):
  image = cv2.imread(f'input1/img_0{i}.jpg')

  # Укажите новые размеры (ширина и высота)
  new_width, new_height = 1024, 768

  # Измените размер изображения
  resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

  # Сохраните измененное изображение
  cv2.imwrite(f'input/img_0{i}.jpg', resized_image)


time: 957 ms (started: 2024-03-21 20:26:12 +00:00)


In [72]:
def keypoints_detection_sift(input_img):
    gray = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT_create()
    kps, dscrs = sift.detectAndCompute(gray, None) #keypoints and descriptors
    return kps, dscrs

time: 1.36 ms (started: 2024-03-21 20:26:21 +00:00)


In [73]:
def keypoints_matching_flann(dscrs1, dscrs2):
    index_params = dict(
        algorithm = 1,
        trees=20
    )
    search_params = dict(checks=20)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    tic = time.time_ns()
    flann_matches = flann.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    good_matches = []
    for neighbour_1, neighbour_2 in flann_matches:
        if neighbour_1.distance < 0.75 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

time: 1.78 ms (started: 2024-03-21 20:26:22 +00:00)


In [74]:
def cut_black_right(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  height, width = gray_image.shape

  # Проход по столбцам изображения справа налево
  for col in range(width - 1, -1, -1):
      # Обработка каждого столбца здесь
      column_data = gray_image[:, col]
      #print(column_data)
      if column_data.sum() > 300:
        rightmost_nonzero_column = col
        break

  # Обрезка изображения справа
  cropped_image = image[:, :rightmost_nonzero_column]
  return cropped_image

time: 1.31 ms (started: 2024-03-21 20:26:25 +00:00)


In [75]:
def cut_black_left(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  height, width = gray_image.shape

  # Проход по столбцам изображения справа налево
  for col in range(0, width-1):
      # Обработка каждого столбца здесь
      column_data = gray_image[:, col]
      #print(column_data)
      if column_data.sum() > 300:
        leftmost_nonzero_column = col
        break

  # Обрезка изображения справа
  cropped_image = image[:, leftmost_nonzero_column:]
  return cropped_image

time: 1.06 ms (started: 2024-03-21 20:26:27 +00:00)


In [76]:
#присоединяем к левой справа
def panorama_right(img1, img2):
#соединяем серединные картинки (1,2) и (2,3)
  ipaths = [img1, img2]
  #ipaths = ["input/img_00.jpg", "input/img_01.jpg", "input/img_02.jpg", "input/img_03.jpg", "input/img_04.jpg"]
  n_imgs = len(ipaths)
  images_info = dict()
  for i_num, ipath in enumerate(ipaths):
      img = plt.imread(ipath)
      images_info[ipath] = {'img': img}
      plt.subplot(1, n_imgs, i_num + 1)
      plt.title(osp.basename(ipath), fontsize=20)
      plt.axis('off')
      plt.imshow(images_info[ipath]['img'])
  plt.show()

  for i_num, ipath in enumerate(images_info):
      img = images_info[ipath]['img']
      keypoints, descriptors = keypoints_detection_sift(img)
      images_info[ipath]['keypoints'] = keypoints
      images_info[ipath]['descriptors'] = descriptors

  matches_images = dict()
  for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
      matches_pair = keypoints_matching_flann(
          images_info[ipath_0]['descriptors'],
          images_info[ipath_1]['descriptors']
      )
      matches_images[(ipath_0, ipath_1)] = matches_pair
  homographies = dict()
  for (ipath_0, ipath_1), matches in matches_images.items():
      src_pts = np.float32([images_info[ipath_0]['keypoints'][m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
      dst_pts = np.float32([images_info[ipath_1]['keypoints'][m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
      M, mask = cv2.findHomography(
          src_pts,
          dst_pts,
          cv2.RANSAC,
          ransacReprojThreshold=3,
          maxIters=20000,
          confidence=0.995
      )
      homographies[ipath_0, ipath_1] = (M, mask.ravel().tolist())

  M_01, _ = homographies[ipaths[1], ipaths[0]]
  img_0, img_1 = images_info[ipaths[0]]['img'], images_info[ipaths[1]]['img']
  # Примените преобразования к изображениям
  img_0_1 = cv2.warpPerspective(img_1, M_01, (img_0.shape[1] + img_1.shape[1], img_1.shape[0]))
  img_0_1[0:img_0.shape[0], 0:img_0.shape[1]] = img_0
  return img_0_1


time: 3.48 ms (started: 2024-03-21 20:26:31 +00:00)


In [77]:
def keypoints_matching_flann2(dscrs1, dscrs2):
    index_params = dict(
        algorithm = 1,
        trees=20
    )
    search_params = dict(checks=10)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    tic = time.time_ns()
    flann_matches = flann.knnMatch(dscrs1, dscrs2, k=2)
    toc = time.time_ns()
    good_matches = []
    for neighbour_1, neighbour_2 in flann_matches:
        if neighbour_1.distance < 0.6 * neighbour_2.distance:
            good_matches.append(neighbour_1)
    return good_matches

time: 1.47 ms (started: 2024-03-21 20:26:40 +00:00)


In [78]:
#присоединяем слева
def panorama_left(img1, img2):
#соединяем серединные картинки (1,2) и (2,3)
  ipaths = [img2, img1]
  #ipaths = ["input/img_00.jpg", "input/img_01.jpg", "input/img_02.jpg", "input/img_03.jpg", "input/img_04.jpg"]
  n_imgs = len(ipaths)
  images_info = dict()
  for i_num, ipath in enumerate(ipaths):
      img = plt.imread(ipath)
      img = cv2.flip(img, 1)
      images_info[ipath] = {'img': img}
      plt.subplot(1, n_imgs, i_num + 1)
      plt.title(osp.basename(ipath), fontsize=20)
      plt.axis('off')
      plt.imshow(images_info[ipath]['img'])
  plt.show()

  for i_num, ipath in enumerate(images_info):
      img = images_info[ipath]['img']
      keypoints, descriptors = keypoints_detection_sift(img)
      images_info[ipath]['keypoints'] = keypoints
      images_info[ipath]['descriptors'] = descriptors

  matches_images = dict()
  for ipath_0, ipath_1 in itertools.permutations(ipaths, 2):
      matches_pair = keypoints_matching_flann(
          images_info[ipath_0]['descriptors'],
          images_info[ipath_1]['descriptors']
      )
      matches_images[(ipath_0, ipath_1)] = matches_pair
  homographies = dict()
  for (ipath_0, ipath_1), matches in matches_images.items():
      src_pts = np.float32([images_info[ipath_0]['keypoints'][m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
      dst_pts = np.float32([images_info[ipath_1]['keypoints'][m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
      M, mask = cv2.findHomography(
          src_pts,
          dst_pts,
          cv2.RANSAC,
          ransacReprojThreshold=3,
          maxIters=20000,
          confidence=0.995
      )
      homographies[ipath_0, ipath_1] = (M, mask.ravel().tolist())

  M_01, _ = homographies[ipaths[1], ipaths[0]]
  img_0, img_1 = images_info[ipaths[0]]['img'], images_info[ipaths[1]]['img']
  # Примените преобразования к изображениям
  img_0_1 = cv2.warpPerspective(img_1, M_01, (img_0.shape[1] + img_1.shape[1], img_1.shape[0]))
  img_0_1[0:img_0.shape[0], 0:img_0.shape[1]] = img_0
  return cv2.flip(img_0_1, 1)


time: 2.66 ms (started: 2024-03-21 20:26:42 +00:00)


In [ ]:
# собираем центр
img_merge_1_2 = cut_black_left(panorama_left("input/img_01.jpg","input/img_02.jpg" ))
plt.imshow(img_merge_1_2)
cv2.imwrite(f'input/img_1_2.jpg', img_merge_1_2)
plt.show()
img_merge_2_3 = cut_black_right(panorama_right("input/img_02.jpg","input/img_03.jpg" ))
plt.imshow(img_merge_2_3)
cv2.imwrite(f'input/img_2_3.jpg', img_merge_2_3)
plt.show()
img_center = cut_black_right(panorama_right("input/img_1_2.jpg","input/img_2_3.jpg" ))
plt.imshow(img_center)
cv2.imwrite(f'input/img_1_3.jpg', img_center)
plt.show()

#работаем с краями
img_merge_left = cut_black_left(panorama_left("input/img_00.jpg","input/img_01.jpg" ))
plt.imshow(img_merge_left)
cv2.imwrite(f'input/img_0_1.jpg', img_merge_left)
plt.show()
img_merge_right = cut_black_right(panorama_right("input/img_03.jpg","input/img_04.jpg" ))
plt.imshow(img_merge_right)
cv2.imwrite(f'input/img_3_4.jpg', img_merge_right)
plt.show()

#цепляем краи к центру
img_merge_0_3 = cut_black_left(panorama_left("input/img_0_1.jpg","input/img_1_3.jpg" ))
plt.imshow(img_merge_0_3)
cv2.imwrite(f'input/img_0_3.jpg', img_merge_0_3)
plt.show()
img_merge_all= cut_black_right(panorama_right("input/img_0_3.jpg","input/img_3_4.jpg" ))
plt.imshow(img_merge_all)
cv2.imwrite(f'input/panorama.jpg', img_merge_all)
plt.show()

# Camera calibration

Для калибровки камер мы будем использовать специальный паттерн, который называется шахматная доска. Его параметры нам хорошо известны, поэтому с его помощью откалибровать камеру будет проще, чем через пайплайн, который мы использовали для построения панорамы.
Изображение chessboard можно найти в поставке opencv на [github](https://github.com/opencv/opencv/blob/4.x/samples/data/chessboard.png)
Его необходимо распечатать и сфотографировать с разных углов и расстояний, чтобы получить достаточное количество изображений для калибровки камеры. Суммарное количество изображений должно быть не менее 10.

In [80]:
input_dir = 'chess'
chessboard_ipaths = sorted(glob.glob(osp.join(input_dir, 'chessboard_*.jpg')))
n_chessboard_imgs = len(chessboard_ipaths)
print(f'Number of chessboard images: {n_chessboard_imgs}')

Number of chessboard images: 12
time: 1.74 ms (started: 2024-03-21 20:29:24 +00:00)


На каждом изображении необходимо найти углы шахматной доски. Для этого используется функция `cv2.findChessboardCorners`. С помощью этой функции мы получим соответствие между углами шахматной доски в реальном мире (object points) и точками на изображении (image points)
После нахождения углов их местоположение можно доуточнить с помощью функции `cv2.cornerSubPix`. Вы можете воспользоваться этой функцией, если вам будет недостаточно получаемого без неё качества калибровки.

In [81]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((6 * 9, 3), np.float32)
objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
# Arrays to store object points and image points from all the images
obj_points = []  # 3d points in real world space
img_points = []  # 2d points in image plane.

time: 1.37 ms (started: 2024-03-21 20:29:27 +00:00)


In [ ]:
_ = plt.figure(figsize=(16, 36))
for i_num, ipath in enumerate(chessboard_ipaths):
    img = cv2.cvtColor(cv2.imread(ipath), cv2.COLOR_BGR2RGB)
    img_grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    result, corners = cv2.findChessboardCorners(img_grayscale, (9, 6), None)
    if result:
        obj_points.append(objp)
        img_points.append(corners)

        cv2.drawChessboardCorners(img, (9, 6), corners, result)
        plt.subplot(math.ceil(n_chessboard_imgs / 3), 3, i_num + 1)
        plt.title(osp.basename(ipath), fontsize=12)
        plt.axis('off')
        plt.imshow(img)
plt.show()

Имея соответствие между углами шахматной доски в реальном мире и точками на изображении, можно калибровать камеру. Для этого используется функция `cv2.calibrateCamera`. В результате калибровки мы получим матрицу камеры и коэффициенты дисторсии.

In [83]:
img = cv2.cvtColor(cv2.imread(chessboard_ipaths[-1]), cv2.COLOR_BGR2RGB)
img_grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
result, mtx, dist, r_vecs, t_vecs = cv2.calibrateCamera(
    objectPoints=obj_points,
    imagePoints=img_points,
    imageSize=img_grayscale.shape[::-1],
    cameraMatrix=None,
    distCoeffs=None
)
print(f'Camera matrix:\n{mtx}')
print(f'Distortion coefficients:\n{dist}')

Camera matrix:
[[3.02309973e+03 0.00000000e+00 1.53128252e+03]
 [0.00000000e+00 3.04337357e+03 1.88011331e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
[[ 3.27355272e-02 -3.32517222e-01 -4.14474257e-03 -2.64174156e-04
   1.78018203e-01]]
time: 361 ms (started: 2024-03-21 20:34:18 +00:00)


In [84]:
np.savetxt('mtx.txt', mtx)
np.savetxt('dist.txt', dist)

time: 2.1 ms (started: 2024-03-21 20:34:22 +00:00)


Эти параметры можно использовать для коррекции изображений, полученных с данной камеры. Для этого используется функция `cv2.undistort`. При этом можно использовать функцию `cv2.getOptimalNewCameraMatrix`, чтобы получить новую матрицу камеры. Она будет содержать только те пиксели, которые содержат информацию об изображении.

In [85]:
rows, cols = img.shape[:2]
new_camera_mtx, roi = cv2.getOptimalNewCameraMatrix(
    cameraMatrix=mtx,
    distCoeffs=dist,
    imageSize=(cols, rows),
    alpha=1,
    newImgSize=(cols, rows)
)
print(f'New camera matrix:\n{new_camera_mtx}')
print(f'Region of interest:\n{roi}')

New camera matrix:
[[2.64848730e+03 0.00000000e+00 1.53336675e+03]
 [0.00000000e+00 2.71988029e+03 1.82087418e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Region of interest:
(177, 105, 2655, 3735)
time: 2.21 ms (started: 2024-03-21 20:34:27 +00:00)


In [ ]:
dst = cv2.undistort(
    src=img,
    cameraMatrix=mtx,
    distCoeffs=dist,
    dst=None,
    newCameraMatrix=new_camera_mtx
)

plt.imshow(dst)
plt.axis('off')
plt.show()

In [ ]:
roi_x, roi_y, roi_w, roi_h = roi
dst = dst[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]
plt.imshow(dst)
plt.axis('off')
plt.show()

In [87]:
!pip install gym==0.25.2
!pip install gym-super-mario-bros==7.4.0
!pip install pyglet==1.5.21
!pip install pygame
import gym
import pygame
from nes_py.wrappers import JoypadSpace #A
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT
#B
env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0")

# Check if 'render_modes' attribute is defined in the environment metadata
if 'render_modes' in env.metadata:
    render_modes = env.metadata['render_modes']
else:
    render_modes = None
    print("Warning: No render modes declared in the environment.")

# Check if 'render_fps' attribute is defined in the environment metadata
if 'render_fps' in env.metadata:
    render_fps = env.metadata['render_fps']
else:
    render_fps = None
    print("Warning: No render fps declared in the environment.")

#env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = JoypadSpace(env, COMPLEX_MOVEMENT) #C
env.metadata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=535717 sha256=5d4015661c846215befecbc1c4b3f4b20e493d45b06cd27a927d90d134fab901
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes-py


/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


{'render.modes': ['rgb_array', 'human'], 'video.frames_per_second': 60}

time: 53.6 s (started: 2024-03-21 21:04:05 +00:00)


In [88]:
gym.__version__
done = True
for step in range(100000):  # D
    #print(step)
    if done:
        state = env.reset()
    env.render()
    state, reward, done, _ = env.step(env.action_space.sample())
    #env.render()

# Close the rendering window after the loop

env.close()
pygame.quit()

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:280: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


ImportError: Library "GLU" not found.

time: 248 ms (started: 2024-03-21 21:05:03 +00:00)
